In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import os
import json
from pipeline import VizPipeline, VizGroup
from params import parse_params
import matplotlib.pyplot as plt
from PIL import ImageOps

from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# A hack to import a local library
sys.path.append("../pilutils/lib/src")
import functions as pilutils

sys.argv=['']; del sys

In [2]:
with open('../partition/ms/train_items_35_1x10-fold.json', 'r') as fp:
    partition_dict = json.load(fp)
    
class_list = partition_dict['metadata']['label_map'].keys()

In [3]:
template_path = '../params.yaml'

params = parse_params(template_path, './params/baseline.yaml')

In [4]:
import numpy as np

@interact(
    inst_idx = widgets.BoundedIntText(
        value=182,
        min=0,
        max=1000,
        step=1,
        description='Instance:',
        disabled=False
    ),
    category = widgets.Dropdown(
        options=sorted(class_list),
        value='race_track',
        description='Category:',
        disabled=False,
    )
)
def demo(inst_idx, category):
    def run(ms3_params):
        param_name_list = [None]
        param_val_list = [None]
        for focal_length in np.arange(0.1, 1.01, 0.2):
            param_name_list.append('focal_length')
            param_val_list.append(focal_length)
        viz_group = VizGroup(ms3_params, 'train', 0, param_name_list=param_name_list, param_val_list=param_val_list)
        img_list = viz_group(category, inst_idx)
        title = category.replace('_', ' ').title()
        row = pilutils.lmosaic(img_list[1:], ['FL={:.1f}m'.format(fl) for fl in param_val_list[1:]], len(img_list[1:]), 1, spacing=0.02, border=3, fontsize=14,
                                box_padding=0.4,
                                box_offset=7, box_type='square', box_alpha=0.8)
        probe = pilutils.lmosaic([img_list[0]], [title], 1, 1, spacing=0.02, border=3, fontsize=16,
                                box_padding=0.4,
                                box_offset=7, box_type='square', box_alpha=0.8)
        full_row = pilutils.hstitch([probe, row], border=2, spacing=10, spacing_color=pilutils.GRAY, border_color=pilutils.GRAY)
        mosaic = ImageOps.expand(full_row, border=3, fill=pilutils.BLACK)
        display(mosaic)

    run(params)

interactive(children=(BoundedIntText(value=182, description='Instance:', max=1000), Dropdown(description='Cate…